In [1]:
import numpy as np
import os

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

from datetime import datetime

import tensorflow as tf
import tfgraphviz as tfg

from sklearn.datasets import fetch_california_housing

from sklearn.preprocessing import StandardScaler

In [2]:
def reset_graph(seed = 42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [3]:
ML_PATH = r"C:\Users\19673418\Desktop\Repository\ADA874"
PROJECT_ROOT_DIR = r"C:\Users\19673418\Desktop\Repository\ADA874\Chapter 9"
CHAPTER_ID = "TensorFlow"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

In [4]:
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

In [5]:
reset_graph()

x = tf.Variable(3, name = "x")
y = tf.Variable(4, name = "y")
f = x*x*y + y + 2

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
tfsession = tf.Session()
tfsession.run(x.initializer)
tfsession.run(y.initializer)
result = tfsession.run(f)
print(result)

42


In [7]:
tfsession.close()

In [8]:
with tf.Session() as tfsession:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()

In [9]:
print(result)

42


In [10]:
tfinit = tf.global_variables_initializer()

In [11]:
with tf.Session() as tfsession:
    tfinit.run()
    result = f.eval()

In [12]:
print(result)

42


In [13]:
tfsession = tf.InteractiveSession()
tfinit.run()
result = f.eval()
print(result)

42


In [14]:
tfsession.close()

In [15]:
reset_graph()

x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [16]:
tfgraph = tf.Graph()
with tfgraph.as_default():
    x2 = tf.Variable(2)

x2.graph is tfgraph

True

In [17]:
x2.graph is tf.get_default_graph()

False

In [18]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x*3

In [19]:
with tf.Session() as tfsession:
    print(y.eval())
    print(z.eval())

10
15


In [20]:
with tf.Session() as tfsession:
    y_val, z_val = tfsession.run([y, z])
    print(y_val)
    print(z_val)

10
15


In [21]:
reset_graph()

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

In [22]:
x = tf.constant(housing_data_plus_bias, dtype = tf.float32, name = "x")
y = tf.constant(housing.target.reshape(-1, 1), dtype = tf.float32, name = "y")
xT = tf.transpose(x)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(xT, x)),xT), y)

In [23]:
with tf.Session() as tfsession:
    theta_value = theta.eval()
    print(theta_value)

[[-3.6959320e+01]
 [ 4.3698898e-01]
 [ 9.4245886e-03]
 [-1.0791138e-01]
 [ 6.4842808e-01]
 [-3.9986235e-06]
 [-3.7866351e-03]
 [-4.2142656e-01]
 [-4.3467718e-01]]


In [24]:
scaler = StandardScaler()

scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

In [25]:
reset_graph()

n_epochs = 1000
learning_rate = 0.01

x = tf.constant(scaled_housing_data_plus_bias, dtype = tf.float32, name = "x")
y = tf.constant(housing.target.reshape(-1, 1), dtype = tf.float32, name = "y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1, 1), name = "theta")

y_pred = tf.matmul(x, theta, name = "predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name = "mse")
gradients = 2/m * tf.matmul(tf.transpose(x), error)

training_op = tf.assign(theta, theta - learning_rate*gradients)

tfinit = tf.global_variables_initializer()

In [26]:
with tf.Session() as tfsession:
    tfsession.run(tfinit)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE = ", mse.eval())
        tfsession.run(training_op)
    best_theta = theta.eval()

Epoch 0 MSE =  12.408014
Epoch 100 MSE =  0.75519687
Epoch 200 MSE =  0.5420874
Epoch 300 MSE =  0.5331699
Epoch 400 MSE =  0.53053814
Epoch 500 MSE =  0.5287963
Epoch 600 MSE =  0.527549
Epoch 700 MSE =  0.52664965
Epoch 800 MSE =  0.5260011
Epoch 900 MSE =  0.52553326


In [27]:
print(best_theta)

[[ 2.0685523e+00]
 [ 8.1063598e-01]
 [ 1.2685776e-01]
 [-2.0784086e-01]
 [ 2.4839850e-01]
 [-1.3083885e-03]
 [-3.9607048e-02]
 [-8.5861266e-01]
 [-8.2600272e-01]]


In [28]:
def my_func(a, b):
    z = 0
    for i in range(100):
        z = a*np.cos(z + 1) + z*np.sin(b - 1)
    return z

In [29]:
reset_graph()

n_epochs = 1000
learning_rate = 0.01

x = tf.constant(scaled_housing_data_plus_bias, dtype = tf.float32, name = "x")
y = tf.constant(housing.target.reshape(-1, 1), dtype = tf.float32, name = "y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1, 1), name = "theta")

y_pred = tf.matmul(x, theta, name = "predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name = "mse")
gradients = tf.gradients(mse, [theta])[0]

training_op = tf.assign(theta, theta - learning_rate*gradients)

tfinit = tf.global_variables_initializer()

In [30]:
with tf.Session() as tfsession:
    tfsession.run(tfinit)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE = ", mse.eval())
        tfsession.run(training_op)
    best_theta = theta.eval()

Epoch 0 MSE =  12.408014
Epoch 100 MSE =  0.75519687
Epoch 200 MSE =  0.5420874
Epoch 300 MSE =  0.53317
Epoch 400 MSE =  0.53053814
Epoch 500 MSE =  0.5287964
Epoch 600 MSE =  0.527549
Epoch 700 MSE =  0.5266497
Epoch 800 MSE =  0.5260011
Epoch 900 MSE =  0.5255332


In [31]:
print(best_theta)

[[ 2.0685525e+00]
 [ 8.1063598e-01]
 [ 1.2685777e-01]
 [-2.0784083e-01]
 [ 2.4839847e-01]
 [-1.3083883e-03]
 [-3.9607048e-02]
 [-8.5861266e-01]
 [-8.2600272e-01]]


In [32]:
reset_graph()

n_epochs = 1000
learning_rate = 0.01

x = tf.constant(scaled_housing_data_plus_bias, dtype = tf.float32, name = "x")
y = tf.constant(housing.target.reshape(-1, 1), dtype = tf.float32, name = "y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1, 1), name = "theta")

y_pred = tf.matmul(x, theta, name = "predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name = "mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)

training_op = optimizer.minimize(mse)

tfinit = tf.global_variables_initializer()

In [33]:
with tf.Session() as tfsession:
    tfsession.run(tfinit)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE = ", mse.eval())
        tfsession.run(training_op)
    best_theta = theta.eval()

Epoch 0 MSE =  12.408014
Epoch 100 MSE =  0.75519687
Epoch 200 MSE =  0.5420874
Epoch 300 MSE =  0.53317
Epoch 400 MSE =  0.53053814
Epoch 500 MSE =  0.5287964
Epoch 600 MSE =  0.527549
Epoch 700 MSE =  0.5266497
Epoch 800 MSE =  0.5260011
Epoch 900 MSE =  0.5255332


In [34]:
print(best_theta)

[[ 2.0685525e+00]
 [ 8.1063598e-01]
 [ 1.2685777e-01]
 [-2.0784083e-01]
 [ 2.4839847e-01]
 [-1.3083883e-03]
 [-3.9607048e-02]
 [-8.5861266e-01]
 [-8.2600272e-01]]


In [35]:
reset_graph()

n_epochs = 1000
learning_rate = 0.01

x = tf.constant(scaled_housing_data_plus_bias, dtype = tf.float32, name = "x")
y = tf.constant(housing.target.reshape(-1, 1), dtype = tf.float32, name = "y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1, 1), name = "theta")

y_pred = tf.matmul(x, theta, name = "predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name = "mse")
optimizer = tf.train.MomentumOptimizer(learning_rate = learning_rate, momentum = 0.9)

training_op = optimizer.minimize(mse)

tfinit = tf.global_variables_initializer()

In [36]:
with tf.Session() as tfsession:
    tfsession.run(tfinit)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE = ", mse.eval())
        tfsession.run(training_op)
    best_theta = theta.eval()

Epoch 0 MSE =  12.408014
Epoch 100 MSE =  0.5252007
Epoch 200 MSE =  0.5243332
Epoch 300 MSE =  0.52432126
Epoch 400 MSE =  0.524321
Epoch 500 MSE =  0.524321
Epoch 600 MSE =  0.524321
Epoch 700 MSE =  0.524321
Epoch 800 MSE =  0.524321
Epoch 900 MSE =  0.5243211


In [37]:
print(best_theta)

[[ 2.068558  ]
 [ 0.8296182 ]
 [ 0.11875144]
 [-0.26552498]
 [ 0.3056947 ]
 [-0.00450307]
 [-0.03932622]
 [-0.89988816]
 [-0.8705434 ]]


In [38]:
reset_graph

A = tf.placeholder(tf.float32, shape = (None, 3))
B = A + 5

In [39]:
with tf.Session() as tfsession:
    B_val1 = B.eval(feed_dict = {A: [[1, 2, 3]]})
    B_val2 = B.eval(feed_dict = {A: [[4, 5, 6], [7, 8, 9]]})

print(B_val1)
print(B_val2)

[[6. 7. 8.]]
[[ 9. 10. 11.]
 [12. 13. 14.]]


In [40]:
reset_graph()

n_epochs = 1000
learning_rate = 0.01
batch_size = 100
n_batches = int(np.ceil(m/batch_size))

x = tf.placeholder(tf.float32, shape = (None, n + 1), name = "x")
y = tf.placeholder(tf.float32, shape = (None), name = "y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1, 1), name = "theta")

y_pred = tf.matmul(x, theta, name = "predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name = "mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)

training_op = optimizer.minimize(mse)

tfinit = tf.global_variables_initializer()

Instructions for updating:
Use tf.cast instead.


In [41]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch*n_batches + batch_index)
    indices = np.random.randint(m, size = batch_size)
    x_batch = scaled_housing_data_plus_bias[indices]
    y_batch = housing.target.reshape(-1, 1)[indices]
    return x_batch, y_batch

In [42]:
#with tf.Session() as tfsession:
#    tfsession.run(tfinit)
#    for epoch in range(n_epochs):
#        for batch_index in range(n_batches):
#            x_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
#            tfsession.run(training_op, feed_dict = {x: x_batch, y: y_batch})
#    best_theta = theta.eval()

In [43]:
#print(best_theta)

In [44]:
reset_graph()

n_epochs = 1000
learning_rate = 0.01

x = tf.constant(scaled_housing_data_plus_bias, dtype = tf.float32, name = "x")
y = tf.constant(housing.target.reshape(-1, 1), dtype = tf.float32, name = "y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1, 1), name = "theta")

y_pred = tf.matmul(x, theta, name = "predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name = "mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)

training_op = optimizer.minimize(mse)

tfinit = tf.global_variables_initializer()

saver= tf.train.Saver()

In [45]:
with tf.Session() as tfsession:
    tfsession.run(tfinit)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE = ", mse.eval())
            save_path = saver.save(tfsession, r"H:\Advanced Data Analytics\Slides\Chapter 9\Chapter9TensorFlowModel.ckpt")
        tfsession.run(training_op)
    best_theta = theta.eval()
    save_path = saver.save(tfsession, r"H:\Advanced Data Analytics\Slides\Chapter 9\Chapter9TensorFlowModelFinal.ckpt")

Epoch 0 MSE =  12.408014
Epoch 100 MSE =  0.75519687
Epoch 200 MSE =  0.5420874
Epoch 300 MSE =  0.53317
Epoch 400 MSE =  0.53053814
Epoch 500 MSE =  0.5287964
Epoch 600 MSE =  0.527549
Epoch 700 MSE =  0.5266497
Epoch 800 MSE =  0.5260011
Epoch 900 MSE =  0.5255332


In [46]:
print(best_theta)

[[ 2.0685525e+00]
 [ 8.1063598e-01]
 [ 1.2685777e-01]
 [-2.0784083e-01]
 [ 2.4839847e-01]
 [-1.3083883e-03]
 [-3.9607048e-02]
 [-8.5861266e-01]
 [-8.2600272e-01]]


In [47]:
with tf.Session() as tfsession:
    saver.restore(tfsession, r"H:\Advanced Data Analytics\Slides\Chapter 9\Chapter9TensorFlowModelFinal.ckpt")
    best_theta_restored = theta.eval()

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from H:\Advanced Data Analytics\Slides\Chapter 9\Chapter9TensorFlowModelFinal.ckpt


In [48]:
print(best_theta_restored)

[[ 2.0685525e+00]
 [ 8.1063598e-01]
 [ 1.2685777e-01]
 [-2.0784083e-01]
 [ 2.4839847e-01]
 [-1.3083883e-03]
 [-3.9607048e-02]
 [-8.5861266e-01]
 [-8.2600272e-01]]


In [49]:
saver = tf.train.Saver({"weights": theta})